In [1]:
# Importação das bibliotecas necessárias
import pandas as pd

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

|---------------------------------------------------------------------|
|                              COLEÇÃO 9 - CLASSES                              |
|-------------------------------------|---------------------------------|-------|
| Categoria                           | Subcategoria                    | ID    |
|-------------------------------------|---------------------------------|-------|
| 3.1. Pastagem                       |                                 | 15    |
| 3.2. Agricultura                    |                                 | 18    |
|                                     | 3.2.1. Lavoura Temporária      | 19    |
|                                     |     3.2.1.1. Soja             | 39    |
|                                     |     3.2.1.2. Cana             | 20    |
|                                     |     3.2.1.3. Arroz (beta)    | 40    |
|                                     |     3.2.1.4. Algodão (beta)  | 62    |
|                                     |     3.2.1.5. Outras Lavouras Temporárias | 41 |
|                                     | 3.2.2. Lavoura Perene          | 36    |
|                                     |     3.2.2.1. Café             | 46    |
|                                     |     3.2.2.2. Citrus           | 47    |
|                                     |     3.2.2.3. Dendê           | 35    |
|                                     |     3.2.1.3. Outras Lavouras Perenes | 48 |
| 3.4. Mosaico de Usos                |                                 | 21    |


Descrição dos códigos: https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2024/08/Legenda-Colecao-9-LEGEND-CODE.pdf

### Ouro

In [4]:
# Cálculo da área total de uso pela agropecuária por Estado
df_agropecuaria = (
                    pd.read_parquet(f'{raiz}/prata_mapbiomas_uso_terra_uf.parquet')
                    .reset_index()
                    .query('classe in (15, 18, 19, 39, 20, 40, 62, 41, 36, 46, 47, 35, 48, 21) & data >= 2000')
                    .assign(fonte = 'Deposição Atmosférica',
                            area = lambda x: x.area/1000000)
                    .groupby(['geocodigo', 'data', 'fonte'])
                    .agg({'area': 'sum'})
                )

Área Destinada a Agropecuária (ha)
Elementos (kg/ha ano)

Cálculo: Cobertura Solo Municipal * (Parâmetro_X/1000)

UF e Brasil = mil t

NPK oriundo de deposição atmosférica incorporado ao solo (t)

In [5]:
parametro_N = 5.5
parametro_P = 0.3
parametro_K = 3.1

In [14]:
df = (df_agropecuaria
      .reset_index()
      .assign(**{
          'N': lambda df: df['area']*(parametro_N/1000),
          'P': lambda df: df['area']*(parametro_P/1000),
          'K': lambda df: df['area']*(parametro_K/1000)
          })
      .drop('area', axis = 1)
      .melt(id_vars=['geocodigo', 'data', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
     .groupby(['geocodigo', 'data', 'fonte', 'elemento'])
     .agg({'valor': 'sum'})
     .reset_index(level='elemento')
     .convert_dtypes()
     .sort_values(by=['elemento', 'geocodigo'],
                   key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2}))
)

In [18]:
df = (
    df
    .rename_axis(index={
        'geocodigo':'state',
        'data': 'date',
        'fonte': 'label'
    })
    .rename(columns={
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_deposicao_atmosferica.csv')